In [2]:
import numpy as np
import tifffile
import napari 
import os
from skimage.measure import regionprops, label
from scipy.spatial.transform import Rotation

from scripts.config_model import save_experiment_config, tree
from scripts.sample_db import SampleDB
from scripts.utils.image_utils import load_tiff_as_hyperstack, save_array_as_hyperstack_tiff

In [3]:
# Function to apply transformation
def apply_transformation(points, transform):
    rotation = Rotation.from_matrix(transform[:3, :3])
    translation = transform[:3, 3]
    return rotation.apply(points) + translation


# Function to check if a point is within the 3D mask
def is_within_mask(point, mask):
    x, y, z = np.round(point).astype(int)
    if 0 <= x < mask.shape[0] and 0 <= y < mask.shape[1] and 0 <= z < mask.shape[2]:
        return mask[x, y, z]
    return False


In [4]:
# Step 1: Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Step 2: Load experiment configuration
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)
print(exp.sample.id)


20220426_RM0008_130hpf_fP1_f3


In [58]:
# Step 3: Load images and transformation matrices

#LM trials images
processed_folder = os.path.join(exp.paths.trials_path, "processed")

lm_trials_images_path = os.path.join(processed_folder, f"sum_raw_trials_{exp.sample.id}.tif")
lm_trials_images =tifffile.imread(lm_trials_images_path)
plane_nr = 1
lm_trial = lm_trials_images[plane_nr,-1]
print(lm_trial.shape)

#LM trials masks
lm_planes_masks_path = os.path.join(exp.paths.trials_path,"masks",f"masks_{exp.sample.id}_params_cp_-1-ft_0.4-st_0.01-resample_True_augment=False.tif")
lm_planes_masks = tifffile.imread(lm_planes_masks_path)

lm_mask = lm_planes_masks[1,-1]
print(lm_mask.shape)

lm_plane_props = regionprops(label(lm_mask))
lm_plane_centroids =  np.array([prop.centroid for prop in lm_plane_props])

#EM warped stack
em_warped_stack_path = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_em_stack_cropped_woResin_rough_rotated_to_LM_fov_lm_res_em.tif')
em_warped_stack = tifffile.imread(em_warped_stack_path)

em_warped_mask_path = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_em_stack_cropped_woResin_rough_rotated_to_LM_fov_lm_res_em_mask_filtered.tif')
em_warped_mask = tifffile.imread(em_warped_stack_path)

(256, 512)
(256, 512)


In [59]:
lm_stack_path =os.path.join(exp.paths.anatomy_path, "raw", f"{exp.sample.id}_anatomyGFRF_001_.tif")
lm_stack = load_tiff_as_hyperstack(lm_stack_path, n_channels=2)


\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\raw\20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif loaded. Shape: (2, 150, 512, 512)


In [60]:
lm_plane_centroids_3d = np.hstack((np.zeros(shape=(lm_plane_centroids.shape[0],1)), lm_plane_centroids[::-1]))
print(lm_plane_centroids_3d)

[[  0.         247.70860927 364.33112583]
 [  0.         211.96571429 318.92      ]
 [  0.         203.55319149 345.08510638]
 [  0.         197.34579439 378.93457944]
 [  0.         196.44444444 321.28395062]
 [  0.         193.19791667 334.40104167]
 [  0.         190.35353535 349.57575758]
 [  0.         187.52554745 391.82481752]
 [  0.         180.13475177 375.10638298]
 [  0.         178.2781457  359.88741722]
 [  0.         178.26368159 398.45273632]
 [  0.         180.11382114 328.57317073]
 [  0.         177.59633028 343.96788991]
 [  0.         174.85849057 168.85849057]
 [  0.         173.0952381  270.69047619]
 [  0.         170.29304029 417.24175824]
 [  0.         170.41935484 386.4562212 ]
 [  0.         164.59390863 290.74111675]
 [  0.         165.90441176 147.64705882]
 [  0.         163.53333333 234.8952381 ]
 [  0.         163.55737705 376.28688525]
 [  0.         161.51219512 161.94308943]
 [  0.         161.73684211 268.99521531]
 [  0.         162.50490196 348.99

In [61]:
# Transformation matrices
t_lm_trial_to_lm_stack_path = os.path.join(processed_folder, f"registration_tform_lm_plane0{plane_nr}_lm_stack.npy")
t_lm_trial_to_lm_stack = np.load(t_lm_trial_to_lm_stack_path)

In [62]:
t_lm_trial_to_lm_stack.shape

(4, 4)

In [63]:
transformed_lm_centroids = apply_transformation(lm_plane_centroids_3d, t_lm_trial_to_lm_stack)

In [73]:
#Visualize
viewer = napari.Viewer()
viewer.add_image(lm_trial, name="lm_trial")
viewer.add_labels(label(lm_mask), name="lm_mask")
viewer.add_image(lm_stack, name="lm_stack")
viewer.add_image(em_warped_stack, name="em_warped_stack")

#viewer.add_points(lm_plane_centroids, name="lm_plane_centroids")
#viewer.add_points(lm_plane_centroids_3d, name="lm_plane_centroids_3d")
viewer.add_points(transformed_lm_centroids, name="transformed_lm_centroids")

<Points layer 'transformed_lm_centroids' at 0x15c437387d0>

In [ ]:
#######################################################

In [1]:
import numpy as np
import pandas as pd
import napari
from scipy.spatial.transform import Rotation
import tifffile

# Function to apply transformation
def apply_transformation(points, transform):
    rotation = Rotation.from_matrix(transform[:3, :3])
    translation = transform[:3, 3]
    return rotation.apply(points) + translation


# Function to check if a point is within the 3D mask
def is_within_mask(point, mask):
    x, y, z = np.round(point).astype(int)
    if 0 <= x < mask.shape[0] and 0 <= y < mask.shape[1] and 0 <= z < mask.shape[2]:
        return mask[x, y, z]
    return False


# Load transformation matrix for lm trials to lm_stack
t_lm_trials_to_lm_stack = np.load("t_lm_trials_to_lm_stack.npy")

# Load centroids of lm plane
lm_trials_centroids = np.load("lm_trials_centroids.npy")

# Load em warped mask
em_warped_mask = tifffile.imread("em_warped_mask.tif")

# Apply transformations to lm_plane_centroids to fit lm_stak and to ultimately fit to em_stack 

centroids_in_lm_stack = apply_transformation(lm_plane_centroids, lm_to_lm_stack)
centroids_in_em_stack = apply_transformation(centroids_in_lm_stack, lm_stack_to_em_stack)


# Identify  centroids within the 3D mask

centroids_in_mask = [point for point in centroids_in_em_stack if is_within_mask(point, em_mask)]


# Validate by visual inspection in napari (Found cells in green)
# Create Napari viewer
viewer = napari.Viewer()

# Add layers to the viewer
viewer.add_image(em_mask, name='EM Mask')
viewer.add_points(points_layer, size=5, face_color='green', name='Found Cells')

# Run Napari viewer
napari.run()


FileNotFoundError: [Errno 2] No such file or directory: 't_lm_trials_to_lm_stack.npy'